In [1]:
import pandas as pd
import datetime as dt
import random as rnd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [21]:
elo_df = pd.read_csv(r'data\elo_data_set.csv', parse_dates=[0])
teams = []
for i in elo_df['team'].unique(): #constructs list of team names
    teams.append(i)
elo_df.head()

,datetime,team,season,opp,day_week,week,W/L,ot,record,home/away,...,opp_pass_yards,opp_rush_yards,opp_turnover,offense_expected_points,defense_expected_points,sp_expected_points,team_elo_before,team_elo_after,opp_elo_before,opp_elo_after
0,2002-09-05 20:38:00,NYG,2002,SFO,Thu,1,0.0,0.0,0-1,HOME,...,166.0,113.0,1.0,-4.72,-0.79,3.46,1500.0,1482.5,1500.0,1517.5
1,2002-09-08 13:04:00,CAR,2002,BAL,Sun,1,1.0,0.0,1-0,HOME,...,212.0,77.0,1.0,-4.38,6.68,4.20,1500.0,1517.5,1500.0,1482.5
2,2002-09-08 13:04:00,CHI,2002,MIN,Sun,1,1.0,0.0,1-0,HOME,...,228.0,140.0,3.0,6.37,-1.20,1.01,1500.0,1517.5,1500.0,1482.5
3,2002-09-08 13:04:00,CLE,2002,KAN,Sun,1,0.0,0.0,0-1,HOME,...,276.0,194.0,1.0,15.18,-17.94,1.15,1500.0,1482.5,1500.0,1517.5
4,2002-09-08 13:04:00,BUF,2002,NYJ,Sun,1,0.0,1.0,0-1,HOME,...,193.0,73.0,1.0,3.16,0.59,-7.81,1500.0,1482.5,1500.0,1517.5


In [3]:
# separates out played games, and the upcoming week.
history_df = elo_df[elo_df['datetime'] < dt.datetime.today()]
future_df = elo_df[elo_df['datetime'] > dt.datetime.today()]
upcoming_week = history_df['week'].iloc[-1] + 1
future_df = future_df[future_df['week'] == upcoming_week]
future_df = future_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', 'W/L']]
print(f'Week to predict : {upcoming_week}')

Week to predict : 8


In [4]:
# Games to be predicted
future_df.reset_index(inplace=True)
future_df.drop('index', axis=1, inplace=True)
future_df

,datetime,team,opp,team_score,opp_score,team_elo_before,opp_elo_before,W/L
0,2020-10-29 20:20:00,CAR,ATL,NaN,NaN,1420.351117,1451.007231,NaN
1,2020-11-01 13:00:00,MIA,LAR,NaN,NaN,1434.532342,1603.925264,NaN
2,2020-11-01 13:00:00,BAL,PIT,NaN,NaN,1685.266570,1629.930316,NaN
3,2020-11-01 13:00:00,CIN,TEN,NaN,NaN,1282.024705,1616.337140,NaN
4,2020-11-01 13:00:00,CLE,LVR,NaN,NaN,1424.263489,1447.441514,NaN
5,2020-11-01 13:00:00,BUF,NWE,NaN,NaN,1537.921187,1610.302984,NaN
6,2020-11-01 13:00:00,KAN,NYJ,NaN,NaN,1743.292439,1318.121744,NaN
7,2020-11-01 13:00:00,GNB,MIN,NaN,NaN,1644.620252,1515.378678,NaN
8,2020-11-01 13:00:00,DET,IND,NaN,NaN,1400.851537,1487.687626,NaN
9,2020-11-01 16:05:00,DEN,LAC,NaN,NaN,1456.027121,1427.976243,NaN


In [12]:
# Functions to simulate games. variables get defined in the next cell
def gameSim():
    team_1_score = (rnd.gauss(team_1_mean_points, team_1_points_std) + rnd.gauss(team_2_opp_mean, team_2_opp_std))/2
    team_2_score = (rnd.gauss(team_2_mean_points, team_2_points_std) + rnd.gauss(team_1_opp_mean, team_1_opp_std))/2
    if int(round(team_1_score)) > int(round(team_2_score)):
        return 1
    elif int(round(team_1_score)) < int(round(team_2_score)):
        return -1
    else: return 0
    
def gamesSim(ns):
    games = []
    team1win = 0
    team2win = 0
    tie = 0
    for i in range(ns):
        gm = gameSim()
        games.append(gm)
        if gm == 1:
            team1win +=1 
        elif gm == -1:
            team2win +=1
        else: tie +=1 
    print(f'{team_1} Win ', team1win/(team1win+team2win+tie),'%')
    print(f'{team_2} Win ', team2win/(team1win+team2win+tie),'%')
    print('Tie ', tie/(team1win+team2win+tie), '%')
    print('------------------')
    return games

In [13]:


for i in range(len(future_df)):
    
    team_1_data = pd.DataFrame()
    team_1_offense = []
    team_1_defense = []
    team_1 = future_df['team'][i]
    team_1_df = history_df[(history_df['team'] == f'{team_1}') | (history_df['opp'] == f'{team_1}')]
    team_1_df = team_1_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', ]]
    team_1_df.reset_index(inplace=True)
    team_1_df.drop('index', axis=1, inplace=True)
    team_counter = 0
    for x in team_1_df['datetime']:
        if team_1_df['team'][team_counter] == f'{team_1}':
            team_1_offense.append(team_1_df['team_score'][team_counter])
            team_1_defense.append(team_1_df['opp_score'][team_counter])
        else:
            team_1_offense.append(team_1_df['opp_score'][team_counter])
            team_1_defense.append(team_1_df['team_score'][team_counter])
        team_counter += 1
    
    team_1_data['offense'] = team_1_offense
    team_1_data['defense'] = team_1_defense
    
    team_2_data = pd.DataFrame()
    team_2_offense = []
    team_2_defense = []
    team_2 = future_df['opp'][i]
    team_2_df = history_df[(history_df['team'] == f'{team_2}') | (history_df['opp'] == f'{team_2}')]
    team_2_df = team_1_df[['datetime', 'team', 'opp', 'team_score', 'opp_score', 'team_elo_before', 'opp_elo_before', ]]
    team_2_df.reset_index(inplace=True)
    team_2_df.drop('index', axis=1, inplace=True)
    team_counter = 0
    for x in team_2_df['datetime']:
        if team_2_df['team'][team_counter] == f'{team_2}':
            team_2_offense.append(team_2_df['team_score'][team_counter])
            team_2_defense.append(team_2_df['opp_score'][team_counter])
        else:
            team_2_offense.append(team_2_df['opp_score'][team_counter])
            team_2_defense.append(team_2_df['team_score'][team_counter])
        team_counter += 1
        
    team_2_data['offense'] = team_2_offense
    team_2_data['defense'] = team_2_defense
    
    team_1_mean_points = team_1_data['offense'].mean()
    team_2_mean_points = team_2_data['offense'].mean()
    team_1_points_std = team_1_data['offense'].std()
    team_2_points_std = team_2_data['offense'].std()

    team_1_opp_mean = team_1_data['defense'].mean()
    team_2_opp_mean = team_2_data['defense'].mean()
    team_1_opp_std = team_1_data['defense'].std()
    team_2_opp_std = team_1_data['defense'].std()
    
    gamesSim(500000)
    #counter += 1

CAR Win  0.498002 %
ATL Win  0.46251 %
Tie  0.039488 %
------------------
MIA Win  0.464058 %
LAR Win  0.496124 %
Tie  0.039818 %
------------------
BAL Win  0.620824 %
PIT Win  0.341792 %
Tie  0.037384 %
------------------
CIN Win  0.487214 %
TEN Win  0.47254 %
Tie  0.040246 %
------------------
CLE Win  0.411644 %
LVR Win  0.54753 %
Tie  0.040826 %
------------------
BUF Win  0.47412 %
NWE Win  0.489128 %
Tie  0.036752 %
------------------
KAN Win  0.55378 %
NYJ Win  0.410476 %
Tie  0.035744 %
------------------
GNB Win  0.624202 %
MIN Win  0.339578 %
Tie  0.03622 %
------------------
DET Win  0.45336 %
IND Win  0.506536 %
Tie  0.040104 %
------------------
DEN Win  0.54814 %
LAC Win  0.413762 %
Tie  0.038098 %
------------------
SEA Win  0.61525 %
SFO Win  0.348606 %
Tie  0.036144 %
------------------
CHI Win  0.512852 %
NOR Win  0.447412 %
Tie  0.039736 %
------------------
PHI Win  0.578386 %
DAL Win  0.383842 %
Tie  0.037772 %
------------------
NYG Win  0.485432 %
TAM Win  0.476

In [14]:
print(len(team_1_offense), ' ', len(team_2_offense))
print(len(team_1_defense), ' ', len(team_2_defense))

308   308
308   308


In [15]:
team_1_data

,offense,defense
0,13.0,16.0
1,26.0,21.0
2,9.0,6.0
3,7.0,21.0
4,21.0,17.0
...,...,...
303,9.0,36.0
304,9.0,17.0
305,34.0,37.0
306,20.0,19.0


In [16]:
team_2_data

,offense,defense
0,16.0,13.0
1,26.0,21.0
2,6.0,9.0
3,7.0,21.0
4,21.0,17.0
...,...,...
303,36.0,9.0
304,9.0,17.0
305,34.0,37.0
306,19.0,20.0


'TAM'

NameError: name 'games' is not defined